In [8]:
#Note to Curtis: While I did initially copy/paste the example, I removed the line drawing (so it just shows the circle
#being tracked without it's trajectory), as well as all the other filler that comes along with that, added your 
#code to checking what quadrent the circle is in during the loop, removed the initial feature finding and instead 
#initialize to x,y=(1,1) and of course allow user to click a point in the imshow to track that point (in draw_circle).
#So I would personally say this is sufficiently transformative, and then once you add it to the class it should be
#fine in terms of being our own work, but I still would leave the credit below, even though it will be clear we
#modified it enough to call it our own

#credit for inspiration and default param values: https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html
import numpy as np
import cv2 as cv
import argparse
import pyautogui

# detects if point c is to the left (counter-clockwise) of the line segment formed by points a and b
def isLeft(a, b, c):
    return ((b["X"] - a["X"])*(c["Y"] - a["Y"]) - (b["Y"] - a["Y"])*(c["X"] - a["X"])) > 0


def draw_circle(event,x,y,flags,param):
    global mouseX,mouseY
    mouseX = -1
    mouseY = -1
    if event == cv.EVENT_LBUTTONDBLCLK:
        cv.circle(img,(x,y),100,(255,0,0),-1)
        mouseX,mouseY = x,y

cap = cv.VideoCapture(0)        
        
cv.namedWindow('frame')
cv.setMouseCallback('frame',draw_circle)

width = int(cap.get(3))
height = int(cap.get(4))
  
#LK parameters
lk_params = dict(winSize  = (15, 15), maxLevel = 2, criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

#Random colors for circles
color = np.random.randint(0, 255, (100, 3))

#initialize gray image
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

#initialize first point to track at arbitary (1,1) pixel
initial_points = np.array([[[1., 1.]]], dtype=np.float32)

#initialize mouse global variables
mouseX = -1
mouseY = -1
current_direction = ""

while(1):
    #Mouse coordinates, global variables
    if mouseX > 0 and mouseY > 0:
        initial_points = np.array([[[mouseX, mouseY]]], dtype=np.float32)

    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    
    #gray current frame
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    #KLT algorithm
    next_points, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, initial_points, None, **lk_params)
    #matching_new becomes the tracked point(s) from next frame
    #matching_old is used to trace a line from the old point(s) to matching_new point(s)
    if next_points is not None:
        matching_new = next_points[st==1]
        matching_old = initial_points[st==1]
    #put circles where tracked points are
    for i, (new, old) in enumerate(zip(matching_new, matching_old)):
        a, b = new.ravel()
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    
    # draw a red line from the upper-left corner to the lower-right corner
    cv.line(frame, (0, 0), (width,height), (255, 0, 0), 3)
    # draw a red line from the upper-right corner to the lower-left corner
    cv.line(frame, (width, 0), (0,height), (255, 0, 0), 3)
    
    cv.imshow('frame', frame)
    #update old frame to be this newest frame, for next iteration
    old_gray = frame_gray.copy()
    #reshape to fit required input for OpenCV LK algorithm
    initial_points = matching_new.reshape(-1, 1, 2)
    #check because sometimes I had an error where initial_points or elements of it were None
    if len(initial_points) > 0:
        if len(initial_points[0][0]) > 1:
            #get coordinates of tracked points (we assume/only care about the first point, as technically
            #the algorithm allows for multiple tracked points, hence the [0][0] part, but the first point
            #will be the tracked point that determines where to turn the snake)
            x_coord = int(initial_points[0][0][0])
            y_coord = int(initial_points[0][0][1])
            
            leftLine = True
            rightLine = True

            leftLine = isLeft({"X": 0, "Y": 0}, {"X": width, "Y": height}, {"X": x_coord, "Y": y_coord})
            rightLine = isLeft({"X": width, "Y": 0}, {"X": 0, "Y": height}, {"X": x_coord, "Y": y_coord})
            #Note to Curtis: I add the "and current_direction !=" check to avoid continuously calling
            #pyautogui, which is an expensive call. I can tell because as I move my tracked finger from
            #one quadrant to another, I see a bit of lag without this check
            if (leftLine and rightLine) and current_direction != "left":
                pyautogui.press("left")
                current_direction = "left"
            elif (not leftLine and not rightLine) and current_direction != "right":
                pyautogui.press("right")
                current_direction = "right"
            elif (leftLine and not rightLine) and current_direction != "down":
                pyautogui.press("down")
                current_direction = "down"
            elif (not leftLine and rightLine) and current_direction != "up":
                pyautogui.press("up")
                current_direction = "up"
    
    keyboard = cv.waitKey(1)
    if keyboard > 0:
        cap.release()
        break
cv.destroyAllWindows()